In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""

import os

# install NeMo
BRANCH = 'main'

GITHUB_ACCOUNT = 'NVIDIA'  # change this if using a fork

# either provide a path to local NeMo repository with NeMo already installed or git clone

# option #1: local path to NeMo repo with NeMo already installed
NEMO_DIR_PATH = "NeMo"

# option #2: download NeMo repo
if 'google.colab' in str(get_ipython()) or not os.path.exists(NEMO_DIR_PATH):
  !git clone -b $BRANCH https://github.com/{GITHUB_ACCOUNT}/NeMo
  %cd NeMo
  !python -m pip install git+https://github.com/{GITHUB_ACCOUNT}/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
  %cd ..

In [ ]:
# If you're not using Colab, you might need to upgrade jupyter notebook to avoid the following error:
# 'ImportError: IProgress not found. Please update jupyter and ipywidgets.'

!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

# Please restart the kernel after running this cell

# Task Description
**Inverse text normalization (ITN)** is an important post-processing step within an automatic speech recognition (ASR) system. 
ITN transforms spoken-domain text into its written form:

> **Input:** "on may third we paid one hundred and twenty three dollars"

> **Output:** "on may 3 we paid \$123".


# Thutmose Tagger approach
We aim to do the following:
1. Align ITN expressions from the [Google Text normalization dataset](https://www.kaggle.com/richardwilliamsproat/text-normalization-for-english-russian-and-polish) on a granular level using [GIZA++](https://github.com/moses-smt/giza-pp), to get a monotonic one-to-one correspondence between each *spoken word* and corresponding *fragments* in written form. 
2. Get a restricted vocabulary of target fragments (tags) that can cover most spoken-written pair conversions.
3. Build training dataset, where the input is the sentence in spoken form and the output is tags for all input words.   
4. Train a token classifier neural model (see Figure below). 
5. Apply a simple postprocessing procedure upon the tag sequence to get the final output

![Thutmose Tagger Architecture](images/thutmose_tagger_architecture.png)

# Dataset

The full English part of [Google Text normalization dataset](https://www.kaggle.com/richardwilliamsproat/text-normalization-for-english-russian-and-polish) consists of 1.1 billion words. For this tutorial we use only small subset of it.


In [ ]:
!wget "https://multilangaudiosamples.s3.us-east-2.amazonaws.com/en_data_small.zip"
!unzip en_data_small

In [ ]:
## actually we do not need separate dev and test data in this tutorial, so just copy it 
!cp -r en_data_small/test en_data_small/dev


The dataset contains unnormalized (i.e. written form) and normalized (i.e. spoken form) sentence pairs that are aligned *on a phrase-level*. The normalized text is synthetic - obtained with the [Kestrel TTS text normalization system](https://www.cambridge.org/core/journals/natural-language-engineering/article/abs/kestrel-tts-text-normalization-system/F0C18A3F596B75D83B75C479E23795DA), so it’s not considered 100% correct.

```
PLAIN   Retrieved       <self>
DATE    18 April 2013   the eighteenth of april twenty thirteen
PUNCT   .       sil
<eos>   <eos>
PLAIN   Neuhorst        <self>
PUNCT   (       sil
PLAIN   Canada  <self>
DATE    2006    two thousand six
PLAIN   Census  <self>
PLAIN   population      <self>
CARDINAL        126     one hundred twenty six
PUNCT   )       sil
PLAIN   is      <self>
PLAIN   a       <self>
PLAIN   small   <self>
PLAIN   hamlet  <self>
PLAIN   in      <self>
PLAIN   Saskatchewan    <self>
PUNCT   ,       sil
PLAIN   Canada  <self>
PLAIN   about   <self>
CARDINAL        30      thirty
PLAIN   minutes <self>
PLAIN   north   <self>
PLAIN   of      <self>
PLAIN   Saskatoon       <self>
PUNCT   .       sil
<eos>   <eos>
```

The following classes appear in the dataset:
* ADDRESS
* CARDINAL
* DATE
* DECIMAL
* DIGIT
* ELECTRONIC
* FRACTION
* LETTERS
* MEASURE
* MONEY
* ORDINAL
* PLAIN
* PUNCT
* TELEPHONE
* TIME
* VERBATIM






# 1. Align ITN expressions on a granular-level

Let's download and compile GIZA++ as we will need it soon

In [ ]:
!git clone https://github.com/moses-smt/giza-pp.git giza-pp
%cd giza-pp
!ls
!make
%cd ..

Do some imports

In [ ]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager
import nemo

import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf
import pandas as pd

First we need to prepare the input data for the aligner (GIZA++).

We regard the corpus of ITN phrase-pairs as a parallel corpus. Parallel means that each pair has an equivalent meaning, though they may consist of any number of tokens and the task of an aligner is to find which source tokens corresponds to which target tokens.
The spoken phrase is tokenized by word boundary, while the written phrase is tokenized as follows: 
1. All alphabetic sequences are separate tokens
2. In numeric sequences each character is a separate token.
3. All non-alphanumeric characters are separate tokens.
4. We add an underscore symbol to mark the beginning and end of a
sequence for future detokenization. 

Example
> **Spoken:** `january thirtieth two thousand five`

> **Written initial:** `jan 30, 2005`

> **Written tokenized**: `_jan_ _3 0 , 2 0 0 5_`

The script [prepare_corpora_for_alignment.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/text_normalization_as_tagging/dataset_preparation/prepare_corpora_for_alignment.py) prepares the described parallel corpora. It extracts all unique ITN phrase-pairs from the Google TN dataset, tokenizes them as described above and stores in separate folders for each semiotic class. It also generates a bash script for running the alignment. At the end it prints how many examples it has found:
```
content/alignment/punct  has  920953  instances
content/alignment/date  has  150499  instances
content/alignment/letters  has  68340  instances
content/alignment/cardinal  has  61029  instances
...
``` 

In [ ]:
WORK_DIR=!pwd  # returns array containing a single path, 
WORK_DIR=WORK_DIR[0]

CORPUS_LANG="en"
if 'google.colab' in str(get_ipython()) or not os.path.exists(NEMO_DIR_PATH):
  NEMO_PATH=WORK_DIR + "/NeMo"
else:
  NEMO_PATH=NEMO_DIR_PATH
GIZA_BIN_DIR=WORK_DIR + "/giza-pp/GIZA++-v2"
MCKLS_BINARY=WORK_DIR + "/giza-pp/mkcls-v2/mkcls"
CORPUS_DIR=WORK_DIR + "/en_data_small"
ALIGNMENT_DIR=WORK_DIR + "/alignment"

!mkdir {ALIGNMENT_DIR}

In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/prepare_corpora_for_alignment.py \
  --data_dir={CORPUS_DIR} \
  --out_dir={ALIGNMENT_DIR} \
  --giza_dir={GIZA_BIN_DIR} \
  --mckls_binary={MCKLS_BINARY} \
  --lang={CORPUS_LANG}

Let's exclude punct class, as our itn task doesn't require to restore punctuation marks

In [ ]:
!rm -r {ALIGNMENT_DIR}/punct


Let's run GIZA++ alignment. 
In this tutorial we only work with three semiotic classes: date, letters and cardinal (in real setting all classes are used, excluding punct).

**Attention**: the environment variable USER should be defined with any value, otherwise GIZA++ ends with segmentation fault. 

In [ ]:
!chmod +x {ALIGNMENT_DIR}/date/run.sh
!chmod +x {ALIGNMENT_DIR}/letters/run.sh
!chmod +x {ALIGNMENT_DIR}/cardinal/run.sh

In [ ]:
## It is necessary to specify environment variable USER=<anything>, otherwise GIZA++ terminates with a segfault 

%cd {ALIGNMENT_DIR}/date
!export USER="user"; ./run.sh
%cd ../..

%cd {ALIGNMENT_DIR}/letters
!export USER="user"; ./run.sh
%cd ../..

%cd {ALIGNMENT_DIR}/cardinal
!export USER="user"; ./run.sh
%cd ../..

GIZA++ will generate many files in our class folders, but we need only two files with final alignments, those with suffixes `A3.final`. The two files correspond to the alignments produced by two GIZA++ runs - direct and reverse (switching source and target corpus). This is a common practice, it allows us to find safer alignment points - tokens that were aligned to one another in both runs. The script [extract_giza_alignments.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/text_normalization_as_tagging/dataset_preparation/extract_giza_alignments.py) heuristically combines these two GIZA++ alignments. It also applies a bunch of regular expressions to correct some alignment mistakes.

In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/extract_giza_alignments.py \
      --mode=itn \
      --giza_dir={ALIGNMENT_DIR}/date \
      --giza_suffix="A3.final" \
      --out_filename=itn.out \
      --lang={CORPUS_LANG}
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/extract_giza_alignments.py \
      --mode=itn \
      --giza_dir={ALIGNMENT_DIR}/letters \
      --giza_suffix="A3.final" \
      --out_filename=itn.out \
      --lang={CORPUS_LANG}
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/extract_giza_alignments.py \
      --mode=itn \
      --giza_dir={ALIGNMENT_DIR}/cardinal \
      --giza_suffix="A3.final" \
      --out_filename=itn.out \
      --lang={CORPUS_LANG}

When we prepared the input corpus of ITN pairs for GIZA++, we made them unique and stored the frequencies in a separate file `freq`. Now let's append the frequencies to the resulting alignments.

In [ ]:
!paste -d"\t" {ALIGNMENT_DIR}/date/freq {ALIGNMENT_DIR}/date/itn.out > {ALIGNMENT_DIR}/date/itn.out2
!paste -d"\t" {ALIGNMENT_DIR}/letters/freq {ALIGNMENT_DIR}/letters/itn.out > {ALIGNMENT_DIR}/letters/itn.out2
!paste -d"\t" {ALIGNMENT_DIR}/cardinal/freq {ALIGNMENT_DIR}/cardinal/itn.out > {ALIGNMENT_DIR}/cardinal/itn.out2

Let's look at what we get. The output should look like
![Top of file with aligned expressions](images/thutmose_tagger_alignment_top.png)
...
![Bottom of file with aligned expressions](images/thutmose_tagger_alignment_bottom.png)


In [ ]:
df = pd.read_csv(ALIGNMENT_DIR + "/cardinal/itn.out2", sep="\t", header=None)
df.columns = ["freq", "verdict", "spoken", "written initial tokens", "left-side alignment", "right-side alignment"]
is_spoken_multiword =  df["spoken"].apply(lambda x: " " in x)
df2 = df[is_spoken_multiword].sort_values("freq", ascending=False).reset_index(drop=True)
df2.head(20)

In [ ]:
df2.tail(10)

# 2. Get a restricted vocabulary of target fragments (tags)

There can be some inconsistencies in the automatic alignments, but nevertheless we now have **one-to-one correspondence** between input words and output fragments. Let's collect all fragments in a vocabulary! The output should look like this
![Tag vocabulary](images/thutmose_tagger_tag_vocabulary.png)



In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/prepare_corpora_after_alignment.py \
  --mode=get_replacement_vocab \
  --giza_dir={ALIGNMENT_DIR} \
  --alignment_filename=itn.out2 \
  --data_dir="" \
  --vocab_filename={WORK_DIR}/replacement_vocab_full.txt \
  --out_filename="" \
  --lang={CORPUS_LANG}


In [ ]:
df = pd.read_csv("replacement_vocab_full.txt.cardinal", sep="\t", header=None)
df.columns = ["replacement tag", "freq"]
df

Tags with low frequencies are likely to be derived from sporadic alignment mistakes, so let's truncate them, and put together the tags from all our semiotic classes.

In [ ]:
!head -n 150 replacement_vocab_full.txt.cardinal > replacement_vocab_cardinal.txt
!head -n 150 replacement_vocab_full.txt.date > replacement_vocab_date.txt
!head -n 150 replacement_vocab_full.txt.letters > replacement_vocab_letters.txt
!cat replacement_vocab_cardinal.txt \
  replacement_vocab_date.txt \
  replacement_vocab_letters.txt > replacement_vocab.select.txt


After concatenation the vocabulary file can contain duplicates of the same tags coming from different semiotic classes, but this is not important at this moment. The final vocabulary will be created later.

In [ ]:
!wc -l replacement_vocab.select.txt

In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/prepare_corpora_after_alignment.py \
  --mode=filter_by_vocab \
  --giza_dir={ALIGNMENT_DIR} \
  --alignment_filename=itn.out2 \
  --data_dir="" \
  --vocab_filename={WORK_DIR}/replacement_vocab.select.txt \
  --out_filename=itn.select.out \
  --lang={CORPUS_LANG}


The script `prepare_corpora_after_alignment.py --mode=filter_by_vocab` discards examples that are not fully covered with our selected replacement vocabulary. We can see that number of lines slightly decreases.
```
4997 content/alignment/cardinal/itn.out2
4681 content/alignment/cardinal/itn.select.out
```

In [ ]:
!wc -l {ALIGNMENT_DIR}/cardinal/itn.out2
!wc -l {ALIGNMENT_DIR}/cardinal/itn.select.out


The format of lines also slightly changes: we add the name of semiotic class, choose only one alignment(left-side or right-side) based on class, and remove unnecessary columns.

![Final alignment](images/thutmose_tagger_final_alignment.png)



In [ ]:
df = pd.read_csv(ALIGNMENT_DIR + "/cardinal/itn.select.out", sep="\t", header=None)
df.columns = ["semiotic class", "spoken", "written initial fragments", "alignment"]
df.head(10)

# 3. Build training dataset 

Now it's time to create a tag-labeled dataset, containing _full sentences_. After previous step we got a large dictionary of ITN phrase conversions *that we know how to tag*. Once again we loop through the Google TN dataset and process each sentence in the following way:

*   If a sentence contains at least one ITN conversion, that is missing from our dictionary, this sentence is discarded.
*   Otherwise we assign tags to the input words
    1.   All words outside ITN conversion spans are tagged as `<SELF>`.
    2.   Tags for words inside ITN spans are taken from the dictionary



In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/prepare_corpora_after_alignment.py \
  --mode=get_labeled_corpus \
  --giza_dir={ALIGNMENT_DIR} \
  --alignment_filename=itn.select.out \
  --data_dir={CORPUS_DIR}/dev \
  --vocab_filename="" \
  --out_filename={CORPUS_DIR}/dev.labeled \
  --lang={CORPUS_LANG}

!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/prepare_corpora_after_alignment.py \
  --mode=get_labeled_corpus \
  --giza_dir={ALIGNMENT_DIR} \
  --alignment_filename=itn.select.out \
  --data_dir={CORPUS_DIR}/train \
  --vocab_filename="" \
  --out_filename={CORPUS_DIR}/train.labeled \
  --lang=${CORPUS_LANG}


The resulting file consists of three columns:
* input words
* target tags
* semiotic spans (if any)

The semiotic spans are separated by semicolon, each span consists of class, begin and end in terms of input word positions e.g. "DATE 6 9".

```
it can be summarized as an error driven transformation based tagger	<SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF>	
this plan was first enacted in nineteen eighty four and continued to be followed for nineteen years	<SELF> <SELF> <SELF> <SELF> <SELF> <SELF> _19 8 4_ <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> _19_ <SELF>	DATE 6 9;CARDINAL 15 16
```
The semiotic spans are used for two purposes:
    
1.   During validation step we calculate accuracy w.r.t. semiotic spans. For example, a DATE span is correct if **all** tag predictions inside this span match the ground truth labels.
2.   The model has additional classification head that predicts a semiotic class label for each of the input words. These predictions are used in the post-processing step for better handling of swaps.



In [ ]:
!head {CORPUS_DIR}/dev.labeled

Get the final label vocabulary, based on our labeled corpora. The output file should look like this
```
KEEP
DELETE
DELETE|_20
DELETE|_19
DELETE|_2
DELETE|_200
DELETE|,20
DELETE|9_
DELETE|9
DELETE|8_
```

In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/dataset_preparation/get_label_vocab.py \
  --train_filename={CORPUS_DIR}/train.labeled \
  --dev_filename={CORPUS_DIR}/dev.labeled \
  --out_filename={CORPUS_DIR}/label_map.txt


In [ ]:
!head {CORPUS_DIR}/label_map.txt

In [ ]:
!echo "ADDRESS" > {CORPUS_DIR}/semiotic_classes.txt
!echo "CARDINAL" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "DATE" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "DECIMAL" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "DIGIT" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "ELECTRONIC" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "FRACTION" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "LETTERS" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "MEASURE" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "MONEY" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "ORDINAL" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "PLAIN" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "PUNCT" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "TELEPHONE" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "TIME" >> {CORPUS_DIR}/semiotic_classes.txt
!echo "VERBATIM" >> {CORPUS_DIR}/semiotic_classes.txt



In [ ]:
!mkdir {WORK_DIR}/datasets

!cp {CORPUS_DIR}/label_map.txt {WORK_DIR}/datasets/label_map.txt
!cp {CORPUS_DIR}/semiotic_classes.txt {WORK_DIR}/datasets/semiotic_classes.txt


Now the file `train.labeled` contains all sentences from initial Google TN data, that we have been able to cover with out tag dictionary. 
From it we can create different datasets for our neural model, trying different sizes and sampling strategies.

Let's create a toy dataset of 5'000 sentences for train set and 5'000 sentences for dev set. Test set is not used - see Evaluation section below.

In [ ]:
DATASET = WORK_DIR + "/datasets/itn_sample10k"
!mkdir {DATASET}
!head -n 5000 {CORPUS_DIR}/train.labeled > {DATASET}/train.tsv
!head -n 5000 {CORPUS_DIR}/dev.labeled > {DATASET}/valid.tsv
!cp {DATASET}/valid.tsv {DATASET}/test.tsv


# 4. Train a token classifier neural model
Now let's run training

In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/normalization_as_tagging_train.py \
  lang=en \
  data.validation_ds.data_path={DATASET}/valid.tsv \
  data.train_ds.data_path={DATASET}/train.tsv \
  model.language_model.pretrained_model_name=bert-base-uncased \
  model.label_map={WORK_DIR}/datasets/label_map.txt \
  model.semiotic_classes={WORK_DIR}/datasets/semiotic_classes.txt \
  trainer.accelerator=gpu \
  trainer.max_epochs=1


Each validation step generates three classification reports where rows correspond to different semiotic classes, `support` column is how many examples of this class occurred in the target of validation set, and `recall` column is the classifier **accuracy** on this class, i.e. percentage of _whole examples_ whose predicted tags match the target.

1.   Tag classification report. `PLAIN` class includes words that are tagged as `<SELF>`.
2.   Tag classification report for **multiword** examples only. They are less trivial and it is harder to achieve high accuracy on them.
3.   Classification report for semiotic classes.

In [ ]:
# the log can be found in nemo_experiments folder
!cat nemo_experiments/training/*/nemo_log_globalrank-0_localrank-0.txt

# Inference

Let's run the inference of our toy model.
First, copy the model, that we've just trained.

In [ ]:
!cp nemo_experiments/training/*/checkpoints/training.nemo .

Generate some input sentences.

In [ ]:
!echo "on the ninth of may four days after her arrival at new orleans west carnifax was decommissioned and returned to the u s s b" > test_sent.txt
!echo "retrieved the fourth of october twenty fifteen" >> test_sent.txt

Run the inference.

In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/normalization_as_tagging_infer.py \
  pretrained_model=./training.nemo \
  inference.from_file=./test_sent.txt \
  inference.out_file=./test_sent.output

In [ ]:
!cat test_sent.output

The inference output consists of 5 columns:

1.   Final output text.
2.   Input text.
3.   Sequence of predicted tags.
4.   Sequence of tags after post-processing (some swaps may be applied).
5.   Sequence of predicted semiotic classes - one class for each input word.

```
on  ninth  may four days after her arrival at new orleans west carnifax was decommissioned and returned to the u s s b	on the ninth of may four days after her arrival at new orleans west carnifax was decommissioned and returned to the u s s b	<SELF> <DELETE> <SELF> <DELETE> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF>	<SELF> <DELETE> <SELF> <DELETE> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF> <SELF>	PLAIN DATE DATE DATE DATE DATE PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN PLAIN LETTERS LETTERS LETTERS LETTERS
retrieved  20  october 20 20	retrieved the fourth of october twenty fifteen	<SELF> <DELETE> _20 <DELETE> <SELF> _20 _20	<SELF> <DELETE> _20 <DELETE> <SELF> _20 _20	PLAIN DATE DATE DATE DATE DATE DATE```


We see that our toy model works and even manages to replace some numbers.

To train a full-fledged model, you need more data.

See also the scripts for the whole pipeline:

> [prepare_dataset_en.sh](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/text_normalization_as_tagging/prepare_dataset_en.sh)

> [normalization_as_tagging_train.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/text_normalization_as_tagging/normalization_as_tagging_train.py)

> [run_infer.sh](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/text_normalization_as_tagging/run_infer.sh)



# Inference with a pretrained model

We can also run inference with a pretrained model [itn_en_thutmose_bert](https://catalog.ngc.nvidia.com/orgs/nvidia/models/itn_en_thutmose_bert).
This is how to use it directly from python.

In [ ]:
thutmose = nemo_nlp.models.ThutmoseTaggerModel.from_pretrained('itn_en_thutmose_bert')
thutmose.summarize()

In [ ]:
from nemo.collections.nlp.data.text_normalization_as_tagging.utils import spoken_preprocessing

lines = ["on the ninth of may four days after her arrival at new orleans west carnifax was decommissioned and returned to the u s s b",
         "retrieved the fourth of october twenty fifteen"]


batch, all_preds = [], []
for i, line in enumerate(lines):
    s = spoken_preprocessing(line)  # this is the same input transformation as in corpus preparation
    batch.append(s.strip())
    outputs = thutmose._infer(batch)
for x in outputs:
    all_preds.append(x)

if len(all_preds) != len(lines):
    raise ValueError(
        "number of input lines and predictions is different: predictions="
        + str(len(all_preds))
        + "; lines="
        + str(len(lines))
    )

for i in range(len(all_preds)):
    print (all_preds[i])

Or we can use the inference script

In [ ]:
!python {NEMO_PATH}/examples/nlp/text_normalization_as_tagging/normalization_as_tagging_infer.py \
  pretrained_model=itn_en_thutmose_bert \
  inference.from_file=./test_sent.txt \
  inference.out_file=./test_sent.output

In [ ]:
!cat test_sent.output